In [21]:
import pandas as pd
import math
from datetime import datetime

<h1>Library to measure volatility actuals</h1>

In [15]:
# Produces a dataframe, with one column per snapshot. A snapshot is a pair [string, dataframe] where the first entry 
# is the name of the snapshot, and the second holds the data for the snapshot. 
#
# The resulting dataframe has 2 levels of column index:
#       -Top level column headers: '# Files', 'K Loc', '# Authors', '# Commits'
#       -Second level column headers: the names of the snapshots
#
# The resulting dataframe aggregates the data of all snapshots, based on the 'by_col' field. Thus, the resulting
# dataframe has one row for each value of the 'by_col' column in the snapshots.
# 
# -snapshots: list of [string, dataframe] pairs. In order to do aggregation, the dataframes are requred to have 
#             these columns: 'Filename', 'Loc', 'Author(s) e-mail', 'CommitId(s)'
#             Additionally, all dataframes must have the a column whose name matches the 'by_col' parameter.
# -by_col: name of the column in the snapshots' dataframes by which stats are sought. For example, if 'Repo' 
#          is used then the stats will be broken by repo, i.e. the output dataframe will have one row per repo.
#
def buildSnapshotStats(snapshots, by_col):
    result_dict = {}
    
    level_1_cols = ['# Files', 'K Loc', '# Authors', '# Commits']
    level_2_cols = list(snapshots.keys())
    
    multicol = pd.MultiIndex.from_product([level_1_cols, level_2_cols])
    
    row_labels = set()
    for snapshot in level_2_cols:
        snapshot_df = snapshots[snapshot]
        if by_col in snapshot_df.columns:
            row_labels = row_labels.union(set(snapshot_df[by_col].unique()))
    
    all_rows = []
    for label in row_labels:
        row = []
        _appendMetricByCounts('Filename'        , row, level_2_cols, snapshots, by_col, label)
        _appendMetricBySumK  ('Loc'           , row, level_2_cols, snapshots, by_col, label)
        _appendMetricByCounts('Author(s) e-mail', row, level_2_cols, snapshots, by_col, label)
        _appendMetricByCounts('CommitId(s)'     , row, level_2_cols, snapshots, by_col, label)
        
        all_rows.append(row)
            
    stats_df = pd.DataFrame(all_rows, index=row_labels, columns=multicol)
            
    return stats_df

# Mutates 'row' by appending the information for column 'col'. 
def _appendMetricByCounts(metric_source_col, row, snapshot_names, snapshots, by_col, row_label):
    for snapshot in snapshot_names:
        snapshot_df = snapshots[snapshot]
        if by_col in snapshot_df.columns and metric_source_col in snapshot_df.columns:
            df = snapshot_df[snapshot_df[by_col] == row_label]
            val = df[metric_source_col].unique().size
            row.append(val)
        else:
            val = None
            row.append(val)
        
# Mutates 'row' by appending the information for column 'col'.
def _appendMetricBySumK(metric_source_col, row, snapshot_names, snapshots, by_col, row_label):
    for snapshot in snapshot_names:
        snapshot_df = snapshots[snapshot]
        if by_col in snapshot_df.columns and metric_source_col in snapshot_df.columns:
            df = snapshot_df[snapshot_df[by_col] == row_label]
            val = df[metric_source_col].sum()/1000
            row.append(val)
        else:
            val = None
            row.append(val)

In [16]:
# Produced a dataframe, with one column per snapshot. A snapshot is a pair [string, dataframe] where the first entry is the
# name of the snapshot, and the second holds the data for the snapshot (i.e., a subset of the rows of the kind of dataframe
# produced by _build_ticket_df)
#
# -by_col: name of the column in the snapshots by which stats are sought. For example, if 'Repo' is used then the stats
#      will be broken by repo.
def buildGranularSnapshotStats(full_repo_df, snapshots, by_col):
    result_dict = {}
    
    level_1_cols = ['# Files', 'K Loc in Repo', 'K Loc change', 'K Loc+', 'K Loc-', '# Authors', '# Commits']
    level_2_cols = list(snapshots.keys())
    
    multicol = pd.MultiIndex.from_product([level_1_cols, level_2_cols])
    
    row_labels = set()
    for snapshot in level_2_cols:
        snapshot_df = snapshots[snapshot]
        if by_col in snapshot_df.columns:
            row_labels = row_labels.union(set(snapshot_df[by_col].unique()))
    
    all_rows = []
    for label in row_labels:
        row = []
        _appendMetricByCounts('Filename'        , row, level_2_cols, snapshots, by_col, label)
        _appendRepoLocK      (full_repo_df,       row,                          by_col, label)
        _appendMetricBySumK  ('Loc'             , row, level_2_cols, snapshots, by_col, label)
        _appendMetricBySumK  ('Loc+'            , row, level_2_cols, snapshots, by_col, label)
        _appendMetricBySumK  ('Loc-'            , row, level_2_cols, snapshots, by_col, label)
        
        _appendMetricByCounts('Author(s) e-mail', row, level_2_cols, snapshots, by_col, label)
        _appendMetricByCounts('CommitId(s)'     , row, level_2_cols, snapshots, by_col, label)
        
        all_rows.append(row)
            
    stats_df = pd.DataFrame(all_rows, index=row_labels, columns=multicol)
    
    for snapshot in level_2_cols:
        multi_col = ('Churn', snapshot)
        stats_df[multi_col] = stats_df[('K Loc change', snapshot)] / stats_df[('K Loc in Repo', snapshot)]
    
    stats_df = stats_df.replace(float('Inf'), 0)
    return stats_df

def _appendRepoLocK(full_repo_df, row, by_col, row_label):
    source_col = 'Loc'
    if by_col in full_repo_df.columns and source_col in full_repo_df.columns:
        df = full_repo_df[full_repo_df[by_col] == row_label]
        row.append(df[source_col].sum()/1000)
    else:
        row.append(None)


<h1>Library to drill into ticket activity</h1>

In [19]:
# Returns a dataframe where there is row for each ticket in the given 'ticket_family'. The returned dataframe
# summarizes basic information about the activity for each ticket, such as the number of files, submodules,
# packages, and artifact families involved in all the commits for the given ticket.
#
# -global_df: a dataframe of commit activity such as what is produced by Git_Analyzer.build_ticket_df
# -ticket_family: a string, corresponding to name of the ticket family for which we seek information.
#
def findTicketFamilyStats(global_df, ticket_family):
    df = global_df[global_df['Ticket(s) Family']==ticket_family]   
    df_byticket = df.groupby('Ticket(s)').apply(_collapseByTicket)
    df_byticket = df_byticket.sort_values(by='# Files', ascending=False)
    df_byticket = df_byticket.reset_index()
    df_byticket = df_byticket.drop('level_1', axis=1)
    return df_byticket
    
def _collapseByTicket(df):
    result_dict = {}
    result_dict['Subject'] = [df['Subject'].iloc[0]]
    result_dict['# Files'] = [df['Filename'].unique().size]
    result_dict['Filenames'] = [list(df['Filename'].unique())]
    result_dict['# Submodules'] = [df['Submodule'].unique().size]
    result_dict['Artifact Families'] = [list(df['Artifact Family'].unique())]
    result_dict['Repo(s)'] = [list(df['Repo'].unique())]
    result_dict['Author(s) e-mail'] = [list(df['Author(s) e-mail'].unique())]
    result_dict['# Packages'] = [df['Package'].unique().size]
    return pd.DataFrame(result_dict)

In [20]:
# Produces a dataframe with exactly one row per ticket in 'full_ticket_df'. The row provides aggregated information
# about the commits associated to that ticket, such as:
#    * the number of repos, commits, files, authors, submodules, packages, artifact families, loc, and dates
#
# -full_ticket_df: dataframe of the kind that is produced by the 'build_ticket_df' function. In particular, it
#  is expected to have columns called:
#
#     * 'Ticket(s)', 'Repo', 'CommitId(s)', 'Filename', 'Author(s) e-mail', 'Submodule', 'Package',
#       'Artifact Family', 'Loc', 'Loc+', 'Loc-', 'Author(s) date'
#
def buildUserStoryScope(full_ticket_df):
    story_scope = full_ticket_df.groupby(['Ticket(s)']).apply(_computeStoryScope)
    story_scope = story_scope.reset_index()
    story_scope = story_scope.drop(['level_1'], axis=1)
    return story_scope

# Helper method invoked by the buildUserStoryScope function
def _computeStoryScope(df):
    result_dict = {}
    result_dict['# Repos']               = [len(df['Repo'].unique())]
    result_dict['# Commits']             = [len(df['CommitId(s)'].unique())]
    result_dict['# Files']               = [len(df['Filename'].unique())]
    result_dict['# Authors']             = [len(df['Author(s) e-mail'].unique())]
    result_dict['# Submodules']          = [len(df['Submodule'].unique())]
    result_dict['# Packages']            = [len(df['Package'].unique())]
    result_dict['# Artifact Families']   = [len(df['Artifact Family'].unique())]
    result_dict['Loc']                   = [df['Loc'].sum()]
    result_dict['Loc+']                  = [df['Loc+'].sum()]
    result_dict['Loc-']                  = [df['Loc-'].sum()]
    
    date_str_min                         = df['Author(s) date'].min()
    date_str_max                         = df['Author(s) date'].max()
    result_dict['First Commit']          = [date_str_min]
    result_dict['Last Commit']           = [date_str_max]
    
    dt_min = datetime.strptime(date_str_min, '%Y-%m-%d')
    dt_max = datetime.strptime(date_str_max, '%Y-%m-%d')
    
    duration = (dt_max - dt_min).days

    result_dict['Duration (days)']     = [duration]
    
    return pd.DataFrame(result_dict)

<h1>Library of DSM (Design Structure Matrix) utilities</h1>

In [18]:
#  Returns two dictionaries of 'square dataframes' (dataframes whose row and column index are the same). Each
#  dataframe basically representing a dependency matrix (DSM) for different measures of interference. 
#  The difference between both dictionaries is that the first one has aggregated values (sums or counts)
#  and the second one's values are drill-down information for the count-based aggregations of the first: the
#  list of elements that were counted.
#
def computeInterferenceDSM(data_df, interference_col, breakout_col, 
                                   aggregate_by_sum, aggregate_by_count, filters):
#  **** EXPLANATION OF FIRST DICTIONARY RETURNED ****
#
#  All dataframes inside this returned dictionary have the same row and column headers, and differ only on 
#  the meaning of the values in them.
#  There is one dataframe for each aggregation metric in the 'aggregate_by_sum' and 'aggregate_by_count' parameters.
#
#  EXAMPLE: Suppose you set:
#              breakout_col       = 'Repo'
#              interference_col   = 'Author(s) e-mail'
#              aggregate_by_sum   = ['Loc']
#              aggregate_by_count = ['Filename', 'CommitId(s)']
#              filters            = {'Artifact Type': ['java']}
#
# This will produce a dictionary 'dict' with 3 entries, with keys 'Loc', 'Filename', 'CommitIds' and the values
# will be square dataframes whose row and column headers are the repos. 'dict['Loc'][x][y]' would contain
# the number of lines of *java* code contributed to repo Y by the team allocated to repo X. I.e., it shows the 
# "interference" that team X has on a repo that is presumably handled by what should be a different team if
# interference were 0. Aggregation is only for *java* code because of the 'filters' parameter.
#     
#  **** EXPLANATION OF SECOND DICTIONARY RETURNED ****
# 
#  This dictionary is similar to the first, except that it only has dataframes for the 'aggregate_by_count'
#  columns, and the values are not the counts, but a list of the elements that were counted.
#
#  **** PARAMETERS ****
#
# -data_df: a dataframe, that must include columns named the same as the 'breakout_col' and 'interference_col'
#           parameters, as well as columns named after the strings in the 'aggregate_by_sum' and 
#           'aggregate_by_count' parameters. In the case of the columns in 'aggregate_by_sum', values in 
#           data_df must be numbers so that they can be added.
# -breakout_col: name of column in 'data_df' used to partition data into groups (the x and y axis of the DSM)
# -interference_col: name of column in 'data_df' whose values, if aggregated per 'breakout_col', are intersected
#                    to determine where there is interference. 
#                    => GOTCHA ALERT: The values in this column must be either strings or lists of strings.
# -aggregate_by_sum: a list of column names for 'data_df' that should be aggregated by adding their values.
#                    => GOTCHA ALERT: If the values under 'interference_col' are lists, this option is disabled.
# -aggregate_by_count: a list of column names for 'data_df' that should be aggregated by adding their values
#                    => GOTCHA ALERT: If the values under 'interference_col' are lists, the only column for 
#                                     which aggregation is allowed is the 'interference_col' itself. Reason:
#                                     when interfering by collections the interference is not a binary
#                                     determination (am I in the set of interfers?) but a messy partial overlap
#                                     (am I intersecting some interfers?)
# -filters: a possibly empty dictionary whose keys are among the column names of 'data_df', and whose values
#           are a list of values for such column that restrict which portion of 'data_df' to aggregate.
#
    #projects_df = data_df.groupby(breakout_col).apply(lambda row: row[interference_col].unique().size)
    projects_df = data_df.groupby(breakout_col).apply(_collapseInterferenceCol, interference_col=interference_col)
    
    projects = list(projects_df.sort_values(ascending=False).index)
    
    x_ticks                     = projects
    y_ticks                     = projects
    main_working_dict           = {} # dict of dicts, from which a dict #1 of dataframes will be built
    drilldown_working_dict      = {} # dict of dicts, from which a dict #2 of dataframes will be built
    for metric in aggregate_by_sum:
        main_working_dict[metric]         = {} # Columns will be the x_ticks and the row index will the y_ticks
    for metric in aggregate_by_count:
        main_working_dict[metric]         = {} # Columns will be the x_ticks and the row index will the y_ticks
        drilldown_working_dict[metric]    = {} # Columns will be the x_ticks and the row index will the y_ticks
    
    result_columns = []
    for x in x_ticks:
        colx                                   = 'By ' + x
        result_columns                         .append(colx)
        for metric in aggregate_by_sum:
            main_working_dict[metric][colx]         = []
        for metric in aggregate_by_count:
            main_working_dict[metric][colx]         = []
            drilldown_working_dict[metric][colx]    = []

        for y in y_ticks:
            dfx                                = data_df[data_df[breakout_col]==x]            
            #x_interferers                      = list(dfx[interference_col].unique())
            x_interferers, interference_type   = _getUniqueValuesOfCol(dfx, interference_col)
            
            dfy                                = data_df[data_df[breakout_col]==y]
            if filters != None:
                for filter_col in filters.keys():
                    dfy = dfy[dfy[filter_col].isin(filters[filter_col])]
            
            # Interference from x in onto y
            #dfy_interfererx                     = dfy[dfy[interference_col].isin(x_interferers)]
            for metric in aggregate_by_sum: 
                if interference_type == list: #This should never happen if caller read the documentation
                    assert(0==1)
                dfy_interfererx                     = dfy[dfy[interference_col].isin(x_interferers)]
                main_working_dict[metric][colx]      .append(dfy_interfererx[metric].sum())
            for metric in aggregate_by_count:   
                if interference_type == str:
                    dfy_interfererx                 = dfy[dfy[interference_col].isin(x_interferers)]
                    things_to_count = list(dfy_interfererx[metric].sort_values().unique())
                else: #Then interference_type = list and metric = interference_col
                    if interference_type == list and metric != interference_col: 
                        #This should never happen if caller read the documentation
                        assert(0==1)
                    things_to_count = _get_XY_intersection(dfy, metric, x_interferers)
                main_working_dict[metric][colx]      .append(len(things_to_count))            
                drilldown_working_dict[metric][colx] .append(things_to_count)            
    
    main_dict                   = {} # dict of DataFrames 
    drilldown_dict              = {} # dict of DataFrames 
    for metric in aggregate_by_sum:   
        main_dict[metric]       = pd.DataFrame(main_working_dict[metric],         
                                               index=y_ticks, columns=result_columns) 
    for metric in aggregate_by_count:   
        main_dict[metric]       = pd.DataFrame(main_working_dict[metric],         
                                               index=y_ticks, columns=result_columns) 
        drilldown_dict[metric]  = pd.DataFrame(drilldown_working_dict[metric],         
                                               index=y_ticks, columns=result_columns) 

    return main_dict, drilldown_dict

def _collapseInterferenceCol(df, interference_col):
    interference_values, interference_type = _getUniqueValuesOfCol(df, interference_col)
    return len(interference_values)

def _getUniqueValuesOfCol(df, col):
    interference_type = str
    values = set()
    for index, row in df.iterrows():
        x = row[col]
        
        # Clean the data in case we loaded an empty string as a nan. Treat it like an empty string, or else
        # subsequent code will think it is a float and throw an assertion for 'Unsupported type...'
        if type(x) == float and math.isnan(x):
            x = ''
        if type(x) == str:
            values.add(x)
        else:
            if type(x) == list:
                values = values.union(set(x))
                interference_type = list
            else: #Don't support other forms of interference, should never get here
                assert 0 == 1, 'Unsupported type ' + str(type(x)) + ' at row index = ' + str(index)
    return list(values), interference_type

def _get_XY_intersection(dfy, col, x_interferers):
    result = set()
    x_set = set(x_interferers)
    for index, row in dfy.iterrows():
        y_set = set(row[col])
        intersection = x_set.intersection(y_set)
        result = result.union(intersection)
    return list(result)